# Running Inference on a Jetson Nano using a custom MXNet model

Sourced from here: https://gluon-cv.mxnet.io/build/examples_detection/demo_webcam.html


In [ ]:
import time
import warnings

import gluoncv as gcv
from gluoncv.utils import try_import_cv2
cv2 = try_import_cv2()
from mxnet import nd, autograd, gluon

In [ ]:
def get_ctx():
    try:
        gpus = mx.test_utils.list_gpus()
        if len(gpus) > 0:
            ctx = []
            for gpu in gpus:
                ctx.append(mx.gpu(gpu))
        else:
            ctx = [mx.cpu()]
    except:
        ctx = [mx.cpu()]
    return ctx

ctx = get_ctx()[0]
ctx

In [ ]:
SHAPE = 512
input_shapes=[('data', (1, 3, SHAPE, SHAPE))]
confidence_threshold = 0.3
CLASSES = ['potatohead']


In [ ]:
# Load the model
# net = gcv.model_zoo.get_model('yolo3_mobilenet1.0_voc', pretrained=True)
# Compile the model for faster speed
# net.hybridize()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    deserialized_net = gluon.nn.SymbolBlock.imports("/home/rob/code/deploy_model_algo_1-symbol.json", ['data'], "/home/rob/code/deploy_model_algo_1-0000.params", ctx=ctx)

In [ ]:
# Load the webcam handler
cap = cv2.VideoCapture(0)
time.sleep(1) ### letting the camera autofocus

In [ ]:
axes = None
NUM_FRAMES = 200 # you can change this
for i in range(NUM_FRAMES):
    # Load frame from the camera
    ret, frame = cap.read()

    # Image pre-processing
    frame = mx.nd.array(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).astype('uint8')
    rgb_nd, frame = gcv.data.transforms.presets.ssd.transform_test(frame, short=512, max_size=700)

    # Run frame through network
    class_IDs, scores, bounding_boxes = deserialized_net(rgb_nd)

    # Display the result
    # img = gcv.utils.viz.cv_plot_bbox(frame, bounding_boxes[0], scores[0], class_IDs[0], class_names=net.classes)
    img = gcv.utils.viz.cv_plot_bbox(frame, bounding_boxes[0], scores[0], class_IDs[0], class_names=CLASSES)
    gcv.utils.viz.cv_plot_image(img)
    cv2.waitKey(1)

In [ ]:
cap.release()
cv2.destroyAllWindows()